<h1>Cour user buy two intem day interval

<h3>1.User buy item1 on day1 and but item 2 on day 6 ,calcualte day1 and day6 interval
<h3>2.Using julianday function to calculate two day interval
<h3>3.Before calculation need to order date and know previous day value.so that can two days value in 

In [3]:
%load_ext sql


In [5]:
%%sql sqlite://
CREATE TABLE buy_item(
    order_id int,
    user_id int,
    login_time date);

INSERT INTO buy_item(order_id,user_id,login_time)values
(1,1,'2025-01-01 10:10:10'),(2,1,'2025-02-01 12:12:12'),(3,1,'2025-03-02 03:40:12'),
(1,2,'2025-01-01 15:15:15'),(5,2,'2025-01-01 07:17:17'),(7,2,'2025-02-01 00:00:00'),
(4,3,'2025-01-01 05:15:13'),(6,3,'2025-01-02 13:15:15'),(1,3,'2025-01-07 15:12:15');


Connecting to 'sqlite://'

9 rows affected.

++
||
++
++

In [43]:
%%sql
with previous as (
select order_id,user_id,login_time,lag(login_time) over (partition by user_id order by login_time) as previous_time from buy_item
),
diff as (
select order_id,user_id,login_time,previous_time,julianday(login_time) as J_login_time,julianday(previous_time) as J_previous_time,
    julianday(login_time)-julianday(previous_time) as diff_day
    from previous

    )
--
select order_id,user_id,previous_time,login_time,diff_day from diff where diff_day not null;

Running query in 'sqlite://'

order_id,user_id,previous_time,login_time,diff_day
2,1,2025-01-01 10:10:10,2025-02-01 12:12:12,31.08474537031725
3,1,2025-02-01 12:12:12,2025-03-02 03:40:12,28.644444444216788
1,2,2025-01-01 07:17:17,2025-01-01 15:15:15,0.33192129619419575
7,2,2025-01-01 15:15:15,2025-02-01 00:00:00,30.36440972238779
6,3,2025-01-01 05:15:13,2025-01-02 13:15:15,1.3333564815111458
1,3,2025-01-02 13:15:15,2025-01-07 15:12:15,5.0812499998137355


In [47]:

%%sql
WITH previous AS (
    SELECT 
        user_id,
        login_time,
        LAG(login_time) OVER (PARTITION BY user_id ORDER BY login_time) AS previous_time
    FROM buy_item
),
diff AS (
    SELECT 
        user_id,
        login_time,
        previous_time,
        JULIANDAY(login_time) - JULIANDAY(previous_time) AS diff_days
    FROM previous
)
SELECT 
    user_id,
    login_time,
    previous_time,
    diff_days,
    ROUND(diff_days * 24, 2) AS diff_hours,
    ROUND(diff_days * 24 * 60, 2) AS diff_minutes
FROM diff where diff_days not null
ORDER BY user_id, login_time;


Running query in 'sqlite://'

user_id,login_time,previous_time,diff_days,diff_hours,diff_minutes
1,2025-02-01 12:12:12,2025-01-01 10:10:10,31.08474537031725,746.03,44762.03
1,2025-03-02 03:40:12,2025-02-01 12:12:12,28.644444444216788,687.47,41248.0
2,2025-01-01 15:15:15,2025-01-01 07:17:17,0.33192129619419575,7.97,477.97
2,2025-02-01 00:00:00,2025-01-01 15:15:15,30.36440972238779,728.75,43724.75
3,2025-01-02 13:15:15,2025-01-01 05:15:13,1.3333564815111458,32.0,1920.03
3,2025-01-07 15:12:15,2025-01-02 13:15:15,5.0812499998137355,121.95,7317.0
